<a href="https://colab.research.google.com/github/kmongsil1105/colab_ipynb/blob/main/AI_YOLOv5x_coco_val2017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5


Cloning into 'yolov5'...
remote: Enumerating objects: 9152, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 9152 (delta 1), reused 3 (delta 0), pack-reused 9143
Receiving objects: 100% (9152/9152), 9.74 MiB | 24.27 MiB/s, done.
Resolving deltas: 100% (6350/6350), done.
/content/yolov5


In [ ]:
! cat requirements.txt

# pip install -r requirements.txt

# Base ----------------------------------------
matplotlib>=3.2.2
numpy>=1.18.5
opencv-python>=4.1.2
Pillow>=7.1.2
PyYAML>=5.3.1
scipy>=1.4.1
torch>=1.7.0
torchvision>=0.8.1
tqdm>=4.41.0

# Logging -------------------------------------
tensorboard>=2.4.1
# wandb

# Plotting ------------------------------------
seaborn>=0.11.0
pandas

# Export --------------------------------------
# coremltools>=4.1  # CoreML export
# onnx>=1.9.0  # ONNX export
# onnx-simplifier>=0.3.6  # ONNX simplifier
# scikit-learn==0.19.2  # CoreML quantization
# tensorflow>=2.4.1  # TFLite export
# tensorflowjs>=3.9.0  # TF.js export

# Extras --------------------------------------
# Cython  # for pycocotools https://github.com/cocodataset/cocoapi/issues/172
# pycocotools>=2.0  # COCO mAP
# albumentations>=1.0.3
thop  # FLOPs computation


In [ ]:
%pip install imgaug==0.2.5

%pip install -qr requirements.txt  # install dependencies

     |████████████████████████████████| 562 kB 5.1 MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.5-py3-none-any.whl size=561438 sha256=6e9f35c274f2c1aa3d93b80e1ff6b9c6f5e42c6672452f69ec372b90d8c4c80f
  Stored in directory: /root/.cache/pip/wheels/60/dd/38/d1dc2cad2b6a66dc0249261004990bccb0f27985c74ba26e49
Successfully built imgaug
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
     |████████████████████████████████| 636 kB 5.4 MB/s 


In [ ]:
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.9.0+cu102 (Tesla K80)


## COCO val2017
Download [COCO val 2017](https://github.com/ultralytics/yolov5/blob/74b34872fdf41941cddcf243951cdb090fbac17b/data/coco.yaml#L14) dataset (1GB - 5000 images), and test model accuracy.

In [ ]:
# Download COCO val2017
torch.hub.download_url_to_file('https://github.com/ultralytics/yolov5/releases/download/v1.0/coco2017val.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../datasets && rm tmp.zip

  0%|          | 0.00/780M [00:00<?, ?B/s]

# 이미지 크기가 640픽셀인 COCO val2017에서 YOLOv5x를 테스트

In [ ]:
#  Run YOLOv5x on COCO val2017 
#  이 명령은 이미지 크기가 640픽셀인 COCO val2017에서 YOLOv5x를 테스트합니다. yolov5x.pt가장 크고 가장 정확한 모델입니다

!python val.py --weights yolov5x.pt --data coco.yaml --img 640 --iou 0.65 --half

val: data=./data/coco.yaml, weights=['yolov5x.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, task=val, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True
YOLOv5 🚀 v5.0-455-g59aae85 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

100% 168M/168M [00:01<00:00, 99.7MB/s]

Fusing layers... 
Model Summary: 476 layers, 87730285 parameters, 0 gradients
val: Scanning '../datasets/coco/val2017' images and labels...4952 found, 48 missing, 0 empty, 0 corrupted: 100% 5000/5000 [00:02<00:00, 1996.05it/s]
val: New cache created: ../datasets/coco/val2017.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 157/157 [10:47<00:00,  4.12s/it]
                 all       5000      36335      0.746      0.626      0.682      0.493
Speed: 0.2ms pre-process, 120.4ms inference, 1.9ms NMS per image at shape (32, 3, 640

In [ ]:
# Download COCO test-dev2017
torch.hub.download_url_to_file('https://github.com/ultralytics/yolov5/releases/download/v1.0/coco2017labels.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../ && rm tmp.zip     # unzip labels
!f="test2017.zip" && curl http://images.cocodataset.org/zips/$f -o $f && unzip -q $f && rm $f  # 7GB,  41k images
%mv ./test2017 ../coco/images              # move to /coco

  0%|          | 0.00/67.7M [00:00<?, ?B/s]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6339M  100 6339M    0     0  47.0M      0  0:02:14  0:02:14 --:--:-- 40.5M


In [ ]:
# Run YOLOv5s on COCO test-dev2017 using --task test

!python val.py --weights yolov5s.pt --data coco.yaml --task test

val: data=./data/coco.yaml, weights=['yolov5s.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=False
YOLOv5 🚀 v5.0-455-g59aae85 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

Fusing layers... 
Model Summary: 224 layers, 7266973 parameters, 0 gradients
Traceback (most recent call last):
  File "/content/yolov5/utils/datasets.py", line 397, in __init__
    raise Exception(f'{prefix}{p} does not exist')
Exception: test: ../datasets/coco/test-dev2017.txt does not exist

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "val.py", line 358, in <module>
    main(opt)
  File "val.py", line 333, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/grad_mode.py", line 28, in decorate_context
    